<a href="https://colab.research.google.com/github/phoudsavanhKongmany/hoc-big-data/blob/main/baitap2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#PHẦN 1: KHỞI TẠO MÔI TRƯỜNG (SETUP)

# 1. Cài đặt PySpark (Chạy trên Google Colab)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark

# 2. Tải dữ liệu mẫu (Romeo & Juliet)
!wget -O romeo.txt https://www.gutenberg.org/cache/epub/1513/pg1513.txt

# 3. Nhân bản dữ liệu để giả lập Big Data (x100 lần)
# File gốc chỉ vài trăm KB, nhân lên để thấy rõ tốc độ xử lý
with open("romeo.txt", "r") as f:content = f.read()
with open("big_romeo.txt", "w") as f:
  for _ in range(100):f.write(content + "\n")

print("✅ Đã chuẩn bị xong dữ liệu: big_romeo.txt")

# 4. Khởi tạo Spark Session
from pyspark.sql import SparkSession
import time

spark = SparkSession.builder \
.appName("EvolutionOfSpark") \
.master("local[*]") \
.getOrCreate()

sc = spark.sparkContext # Lấy SparkContext để dùng cho RDD cũ
print("🚀 Spark đã sẵn sàng!")

--2026-01-13 14:57:52--  https://www.gutenberg.org/cache/epub/1513/pg1513.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169546 (166K) [text/plain]
Saving to: ‘romeo.txt’

romeo.txt           100%[===================>] 165.57K  --.-KB/s    in 0.03s   

2026-01-13 14:57:52 (5.06 MB/s) - ‘romeo.txt’ saved [169546/169546]

✅ Đã chuẩn bị xong dữ liệu: big_romeo.txt
🚀 Spark đã sẵn sàng!


In [4]:
#PHẦN 2: CÁCH 1 - GIẢI BẰNG RDD (TƯ DUY MAPREDUCE CŨ)

print("⏳ Đang chạy RDD (Cách cũ)...")
start_time = time.time()

# 1. Đọc file vào RDD
rdd_text = sc.textFile("big_romeo.txt")

# 2. Thực hiện MapReduce
# - flatMap: Cắt dòng thành từ
# - map: Gán số 1 cho mỗi từ -> (word, 1)
# - reduceByKey: Cộng dồn các số 1 lại
rdd_counts = rdd_text.flatMap(lambda line: line.split(" ")) \
.map(lambda word: (word, 1)) \
.reduceByKey(lambda a, b: a + b)

# 3. Sắp xếp (Sort) - Việc này rất tốn kém với RDD
# Phải đảo ngược (count, word) để sort theo key là count
rdd_sorted = rdd_counts.map(lambda x: (x[1], x[0])) \
.sortByKey(ascending=False)

# 4. Lấy kết quả (Action)
result_rdd = rdd_sorted.take(10)

end_time = time.time()
print(f"⏱️ Thời gian chạy RDD: {end_time - start_time:.4f} giây")
print("Top 10 từ (RDD):", result_rdd)

⏳ Đang chạy RDD (Cách cũ)...
⏱️ Thời gian chạy RDD: 10.6638 giây
Top 10 từ (RDD): [(168900, ''), (78000, 'the'), (55100, 'I'), (54100, 'and'), (52400, 'to'), (47600, 'of'), (45800, 'a'), (35000, 'in'), (31300, 'is'), (30400, 'my')]


In [5]:
#PHẦN 3: CÁCH 2 - GIẢI BẰNG DATAFRAME (TƯ DUY CẤU TRÚC)

from pyspark.sql.functions import split, explode, col, desc

print("⏳ Đang chạy DataFrame (Cách mới)...")
start_time = time.time()

# 1. Đọc file vào DataFrame (Có cấu trúc dòng/cột)
df_text = spark.read.text("big_romeo.txt")

# 2. Xử lý
# - split: Tách chuỗi
# - explode: Biến mảng thành các dòng riêng biệt
df_counts = df_text.select(explode(split(col("value"), " ")).alias("word")) \
.groupBy("word") \
.count() \
.orderBy(desc("count"))

# 3. Lấy kết quả
result_df = df_counts.limit(10).collect()

end_time = time.time()
print(f"⏱️ Thời gian chạy DataFrame: {end_time - start_time:.4f} giây")
print("------------------------------------------------")
df_counts.show(10)

⏳ Đang chạy DataFrame (Cách mới)...
⏱️ Thời gian chạy DataFrame: 8.6006 giây
------------------------------------------------
+----+------+
|word| count|
+----+------+
|    |168900|
| the| 78000|
|   I| 55100|
| and| 54100|
|  to| 52400|
|  of| 47600|
|   a| 45800|
|  in| 35000|
|  is| 31300|
|  my| 30400|
+----+------+
only showing top 10 rows


In [6]:
#PHẦN 4: CÁCH 3 - GIẢI BẰNG SPARK SQL (TƯ DUY QUEN THUỘC)

print("⏳ Đang chạy Spark SQL...")
start_time = time.time()

# 1. Biến DataFrame ban đầu thành bảng tạm (Temp View)
# Nhớ lại bước trước: df_text chứa 1 cột là "value"
df_text.createOrReplaceTempView("raw_book")

# 2. Viết SQL
# Logic: Split xong Explode ngay trong câu SQL (Spark hỗ trợ rất mạnh)
query = """
SELECT word, count(*) as total
FROM (
SELECT explode(split(value, ' ')) as word
FROM raw_book
)
GROUP BY word
ORDER BY total DESC
"""

# 3. Thực thi
df_sql = spark.sql(query)
result_sql = df_sql.limit(10).collect()

end_time = time.time()
print(f"⏱️ Thời gian chạy SQL: {end_time - start_time:.4f} giây")
df_sql.show(10)

⏳ Đang chạy Spark SQL...
⏱️ Thời gian chạy SQL: 2.0049 giây
+----+------+
|word| total|
+----+------+
|    |168900|
| the| 78000|
|   I| 55100|
| and| 54100|
|  to| 52400|
|  of| 47600|
|   a| 45800|
|  in| 35000|
|  is| 31300|
|  my| 30400|
+----+------+
only showing top 10 rows
